In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR 
from sklearn.preprocessing import StandardScaler

import time

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.data import random_split
import random, os

In [ ]:
#Ensure data reproducibility
def random_seed(seed):
    random.seed(seed)

    os.environ['PYTHONHASHSEED'] =str(seed)

    np.random.seed(seed)

    torch.manual_seed(seed)

    torch.cuda.manual_seed(seed)

    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic =True
    
#Customize functions
#Define functions
#Standardization
def ss(features, labels):
    scaler = StandardScaler()
    X_s = scaler.fit_transform(features)
    X_s = pd.DataFrame(X_s)
    data = pd.concat([X_s, labels], axis=1)
    return data

def model_score(model, x, y, trainsize, testsize):
    cv = ShuffleSplit(n_splits=10, train_size=trainsize, test_size=testsize, random_state=0)
    rmse = cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv=cv)
    rmse_score = np.sqrt(-rmse)
    rmse_mean = rmse_score.mean()

    mae = cross_val_score(model, x, y, scoring="neg_mean_absolute_error", cv=cv)
    mae_score = -mae
    mae_mean = mae_score.mean()

    r2 = cross_val_score(model, x, y, scoring='r2', cv=cv)
    r2_mean = r2.mean()

    scores = [rmse_score, rmse_mean, mae_score, mae_mean, r2, r2_mean]

    rmse = pd.DataFrame(scores[0], columns=['rmse'], index = [np.arange(len(scores[0]))])
    mae = pd.DataFrame(scores[2], columns=['mae'], index = [np.arange(len(scores[2]))])
    R2 = pd.DataFrame(scores[4], columns=['R2'], index = [np.arange(len(scores[4]))])
    scores_df = pd.concat([rmse,mae,R2], axis=1)
    return scores_df


def ToCsv(model, Xtest, ytest, filename):
    ytest = pd.DataFrame(ytest.values, index=[np.arange(len(ytest))], columns=['yreal1', 'yreal2', 'yreal3'])
    ypredict = model.predict(Xtest)
    ypredict = pd.DataFrame(ypredict, index=[np.arange(len(ytest))], columns=['ypredict1', 'ypredict2', 'ypredict3'])
    # ypredict
    output = pd.concat([ytest, ypredict], axis=1)
    output.to_csv(filename)

def DataProcess(path):
    data = pd.read_csv(path)
    data_df = pd.DataFrame(data)
    X_df = data_df.iloc[:,1:6]
    y_df = data_df.iloc[:,6:9]

    data_s = ss(X_df,y_df)
    X = data_s.iloc[:,0:5]
    y = data_s.iloc[:,5:]
    return X, y

def DataSplit(X,y, testsize, seed):
    random_seed(seed)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=testsize)
    return Xtrain, Xtest, ytrain, ytest

In [ ]:
#DNN model

def Df2Tensor(df):
    array = np.array(df)
    tensor = torch.tensor(array, dtype=torch.float32)
    return tensor

def ToDataset(*args):
    return TensorDataset(*args)

def ToDataLoader(dataset, batchsize):
    return DataLoader(dataset, batchsize, shuffle=True)


class Net(nn.Module):
    def __init__(self, 
            input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4, 
            dropout1, dropout2, dropout3, dropout4):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim,hidden_layer1)
        self.layer2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.layer3 = nn.Linear(hidden_layer2,hidden_layer3)
        self.layer4 = nn.Linear(hidden_layer3,hidden_layer4)
        self.layer5 = nn.Linear(hidden_layer4,output_dim)

        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
        self.dropout3 = nn.Dropout(dropout3)
        self.dropout4 = nn.Dropout(dropout4)


    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = F.relu(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = F.relu(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        return x


class Metrics():
    def __init__(self, net, dataloader):
        dataset = dataloader.dataset
        self.features = dataset[:][0]
        self.labels = dataset[:][1]
        self.y_hat = torch.clamp(net(self.features), 1, float('inf'))
    def rmse(self):
        return torch.sqrt(F.mse_loss(self.y_hat, self.labels))
    def mae(self):
        return F.l1_loss(self.y_hat, self.labels)
    def smooth_mae(self):
        return F.smooth_l1_loss(self.y_hat, self.labels)
    def r2(self):
        # return r2_score(self.labels.detach().numpy(), self.y_hat.detach().numpy()) 
        SS_res = torch.sum(torch.square(self.labels-self.y_hat))
        SS_tot = torch.sum(torch.square(self.labels - torch.mean(self.labels)))
        r2 = 1 - SS_res / SS_tot
        return r2 

def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, std=0.01)

def DataConcat(Xtrain, Xtest, ytrain, ytest):
    train_df = [Xtrain, ytrain]
    test_df = [Xtest, ytest]
    train_data = pd.concat(train_df,axis=1)
    test_data = pd.concat(test_df,axis=1)
    return train_data, test_data

from torch.optim.lr_scheduler import StepLR

def train(net, dataloader, loss, num_epochs, lr, wd):
    net.train()
    
    optimizer = torch.optim.Adam(net.parameters(), lr = lr, weight_decay = wd)
    scheduler = StepLR(optimizer, step_size=num_epochs/3, gamma=0.3)

    for epoch in range(num_epochs):
        for X, y in dataloader:
            optimizer.zero_grad()
            
            l = loss(net(X), y) 
            l.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()


def NetEval(net, dataloader, num_epochs, loss, lr, wd):
    eval_list = []
    rmse, mae, r2 = [], [], []
    for epochs in range(num_epochs):
        net.train()
        train(net, dataloader, loss, num_epochs, lr, wd)

        net.eval()
        test_metrics = Metrics(net, dataloader)


        rmse.append(test_metrics.rmse().detach().item())
        mae.append(test_metrics.mae().detach().item())
        r2.append(test_metrics.r2().detach().item())
    return r2, mae, rmse

In [ ]:
#Define hyperparameters and the network
input_dim, output_dim, hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4 = 5, 3, 120,60,30,15
num_epochs, lr, wd, batch_size = 1000, 0.01, 0, 54
dropout1, dropout2, dropout3, dropout4 = 0,0,0,0

loss = nn.MSELoss()

net = Net(input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4,
            dropout1, dropout2, dropout3, dropout4)
net.apply(init_weights)

Net(
  (layer1): Linear(in_features=5, out_features=120, bias=True)
  (layer2): Linear(in_features=120, out_features=60, bias=True)
  (layer3): Linear(in_features=60, out_features=30, bias=True)
  (layer4): Linear(in_features=30, out_features=15, bias=True)
  (layer5): Linear(in_features=15, out_features=3, bias=True)
  (dropout1): Dropout(p=0, inplace=False)
  (dropout2): Dropout(p=0, inplace=False)
  (dropout3): Dropout(p=0, inplace=False)
  (dropout4): Dropout(p=0, inplace=False)
)

In [ ]:
#FEA data
Path = "FEA_data.csv"

seed = 0
trainsize, testsize = 0.1,0.9

X, y = DataProcess(Path)
Xtrain, Xtest, ytrain, ytest = DataSplit(X,y, testsize, seed)

In [ ]:
# type(Xtrain)
x1 = Xtrain.to_numpy()
y1 = ytrain.to_numpy()


In [ ]:
train_dataset = ToDataset(Df2Tensor(Xtrain), Df2Tensor(ytrain))
test_dataset = ToDataset(Df2Tensor(Xtest), Df2Tensor(ytest))

batchsize = 54
train_dataloader = ToDataLoader(train_dataset, batchsize)
test_dataloader = ToDataLoader(test_dataset, batchsize)

In [ ]:
#DNN training
train(net, train_dataloader, loss, num_epochs, lr, wd)

In [ ]:
eval_epochs = 10
wd = 0
r2, mae, rmse = NetEval(net, test_dataloader, eval_epochs, loss, lr, wd)
DNN_r2_mean = np.mean(r2)
DNN_mae_mean = np.mean(mae)
DNN_rmse_mean = np.mean(rmse)
print('The average R² of DNN:{}, the average MAE:{}, the average RMSE:{}'.format(DNN_r2_mean, DNN_mae_mean, DNN_rmse_mean))
all_evals = [
             [DNN_r2_mean, DNN_mae_mean, DNN_rmse_mean]
             ]

DNN r2平均值:0.9791010320186615, mae平均值:3.378544545173645, rmse平均值:4.57615213394165


In [ ]:
#Physics-informed loss function
import math
def CuLoss(g, rpm, rcoil):
    g = g/1000
    rcoil = rcoil/1000
    rpm = rpm/1000 
    zcoil = 8e-3     
    Jm = 4e6          
    N = 576            
    p = 4            

    Rm1 = 8e-3         
    Rm2 = Rm1 + rpm    
    Rs1 = Rm2 + g      

    ks = 0.8           
    sd = ks * rcoil * zcoil / N 
    R = N * 2 * torch.pi * (Rs1 + rcoil / N) / sd  
    
    cul = (Jm * sd)**2 * R / p
    cul = cul / 1e3  

    return cul/1000

import scipy.special as sp
from scipy import integrate

def Avgforce(z, ap, rpm, g, rcoil, b0):
    rpm = rpm / 1000
    g = g / 1000
    rcoil = rcoil / 1000
    b0 = b0 / 1000
  
    Rs1 = 17e-3 
    Rs = 33.9e-3  
    Rr = 8e-3  
    Rm = Rr + rpm  
    tp = 36e-3  
    tm = 36e-3 * ap  
    Brem = 1.23000000140598 
    u0 = 4e-7 * math.pi  
    ur = 1.0997785406  
    tcoil = 7.8e-3  
    Jrms = 4e6  
    ts = 12e-3 
    p = 4  
    SAP = rcoil * tcoil  
    Nsp = 100 
    Pf = 1  
    
    for value in z:
            n = 10  
    Fn1 = []  
    Fn3 = []  
    for i in range(1, n+1):

        mn = (2 * n - 1) * math.pi / tp 
        Pn = 4 * Brem / tp * math.sin((2 * n - 1) * math.pi / 2 * ap)  
        
        c1n = sp.i0(mn * Rs)  
        c2n = sp.k0(mn * Rs)  
        c3n = sp.i0(mn * Rr)  
        c4n = sp.k0(mn * Rr)  
        c5n = sp.i0(mn * Rm)  
        c6n = sp.k0(mn * Rm)  
        c7n = sp.i1(mn * Rm)  
        c8n = sp.k1(mn * Rm)  

        def fan(x):
            return sp.k1(x) / (sp.i1(x) * sp.k0(x) + sp.k1(x) * sp.i0(x))  

        def fbn(x):
            return sp.i1(x) / (sp.i1(x) * sp.k0(x) + sp.k1(x) * sp.i0(x))  

        def FAn(r):
            return Pn / mn * integrate.quad(fan, mn * Rr, mn * r)[0] 

        def FBn(r):
            return Pn / mn * integrate.quad(fbn, mn * Rr, mn * r)[0]  

        A = torch.tensor([[ur * (c5n / c6n - c1n / c2n), -(c5n / c6n - c3n / c4n)],
                          [(c7n / c8n + c1n / c2n), -(c7n / c8n + c3n / c4n)]])
        B = torch.tensor([FAn(Rm) * c5n / c6n + FBn(Rm),
                          FAn(Rm) * c7n / c8n - FBn(Rm)])

        X = torch.linalg.solve(A, B)  

        a1n = X[0] 
        b1n = c1n / c2n * a1n  

        g1 = g + rpm / ur 
        k1 = b0 / (2 * g1)  
        gamma = 4 / math.pi * (k1 * math.atan(k1) - math.log(1 + k1 ** 2))  
        Kc = ts / (ts - gamma * g1)  

        ge = g + (Kc - 1) * g1  
        rse = Rm + ge  

        bs0 = 1e-3  
        Krn = rse * (a1n * sp.i1(mn * rse) + b1n * sp.k1(mn * rse))  
        Kpn = math.sin(mn * ts / 2)  
        tsp = ts - tcoil  
        Kdn = math.sin(Nsp * mn * tsp / 2) / (Nsp * math.sin(mn * tsp / 2)) * math.sin(mn * b0 / 2) / (mn * bs0 / 2)  
        Kdpn = Kdn * Kpn  
        kn = 4 * math.pi * p * Kdpn * Krn * SAP  

        if i == 1:
            F1 = (-math.sqrt(2) * kn * 1.5 * Jrms * Pf * 1e3 * ((rpm * 1e3 - 1.8) * 0.88) + bs0 * 1e3 * math.sin(math.pi / 3)) \
                * (math.exp(ap) / math.exp(0.5)) * ((rcoil + 2e-3) * 1e3 / 100 + 1)

        if i % 3 == 1 and i >= 3:
            fn1 = math.sqrt(2) * Jrms * 1.5 * kn * (-1) ** i  
            Fn1.append(fn1 * math.cos((2 * i - 2) * (math.pi * value) / tp) * 1e3 / 3 / 10)

            
        elif i % 3 == 0:
            fn3 = math.sqrt(2) * Jrms * 1.5 * kn * (-1) ** (i + 1) 
            Fn3.append(fn3 * math.cos((2 * i - 5) * (math.pi * value) / tp) * 1e3 / 3 / 10)
            
            
    avgforce = -(F1 + torch.sum(torch.tensor(Fn1)) + torch.sum(torch.tensor(Fn3))) 
    avgforce = abs(avgforce)
    if avgforce>=150:
        avgforce = avgforce * 0.69
    
    return avgforce

def Fluforce(ap, rpm, g, rcoil, b0):
    rpm = rpm / 1000
    g = g / 1000
    rcoil = rcoil / 1000
    b0 = b0 / 1000

    Rs1 = 17e-3          
    Rs = 33.9e-3         
    Rr = 8e-3           
    Rm = Rr + rpm        
    tp = 36e-3          
    tm = 36e-3 * ap     
    Brem = 1.23000000140598  
    u0 = 4e-7 * torch.pi 
    ur = 1.0997785406   
    tcoil = 7.8e-3       
    Jrms = 4e6           
    ts = 12e-3           
    p = 4                
    SAP = rcoil * tcoil  
    Nsp = 100            
    Pf = 1              
 
    
    K1 = 0
    Ksum = 0

    n = 10  
    for i in range(1, n + 1):
      
        mn = (2 * n - 1) * math.pi / tp  
        Pn = 4 * Brem / tp * math.sin((2 * n - 1) * math.pi / 2 * ap)  
    
        c1n = sp.i0(mn * Rs) 
        c2n = sp.k0(mn * Rs)  
        c3n = sp.i0(mn * Rr)  
        c4n = sp.k0(mn * Rr)  
        c5n = sp.i0(mn * Rm) 
        c6n = sp.k0(mn * Rm)  
        c7n = sp.i1(mn * Rm)  
        c8n = sp.k1(mn * Rm)  

        
        def fan(x):
            return sp.k1(x) / (sp.i1(x) * sp.k0(x) + sp.k1(x) * sp.i0(x)) 

        def fbn(x):
            return sp.i1(x) / (sp.i1(x) * sp.k0(x) + sp.k1(x) * sp.i0(x))  

    
        def FAn(r):
            return Pn / mn * integrate.quad(fan, mn * Rr, mn * r)[0]

        def FBn(r):
            return Pn / mn * integrate.quad(fbn, mn * Rr, mn * r)[0]

       
        A = torch.tensor([[ur * (c5n / c6n - c1n / c2n), -(c5n / c6n - c3n / c4n)],
                          [(c7n / c8n + c1n / c2n), -(c7n / c8n + c3n / c4n)]])
        B = torch.tensor([FAn(Rm) * c5n / c6n + FBn(Rm),
                          FAn(Rm) * c7n / c8n - FBn(Rm)])
        
      
        X = torch.linalg.solve(A, B)
        a1n = X[0]
        b1n = c1n / c2n * a1n

        
        g1 = g + rpm / ur  
        k1 = b0 / (2 * g1)  
        gamma = 4 / math.pi * (k1 * math.atan(k1) - math.log(1 + k1**2))  
        Kc = ts / (ts - gamma * g1)  
        
       
        ge = g + (Kc - 1) * g1
        rse = Rm + ge
        
        
        bs0 = 1e-3  
        Krn = rse * (a1n * sp.i1(mn * rse) + b1n * sp.k1(mn * rse))  
        Kpn = math.sin(mn * ts / 2) 
        tsp = ts - tcoil  
        Kdn = math.sin(Nsp * mn * tsp / 2) / (Nsp * math.sin(mn * tsp / 2)) * math.sin(mn * b0 / 2) / (mn * bs0 / 2)  
        Kdpn = Kdn * Kpn  
        kn = 4 * math.pi * p * Kdpn * Krn * SAP  
        
        
        
        if i == 1:
            K1 = K1 + kn
        else:
            Ksum = Ksum + kn
    
    pkforce = math.sqrt(Ksum**2) / K1 * (math.sqrt(2) * kn * 1.5 * Jrms * Pf * 1e3 * ((rpm * 1e3 - 1.8) * 0.88) + bs0 * 1e3 * math.sin(math.pi / 3)) * \
              (math.exp(ap) / math.exp(0.5)) * ((rcoil + 2e-3) * 1e3 / 100 + 1) / p**2
    if pkforce>=33:
        pkforce = pkforce*0.58
    return pkforce

def Parameters():
    data = {}
    data['Rs1'] = 17e-3
    data['Rs'] = 33.9e-3
    data['Rm'] = 16e-3
    data['Rr'] = 8e-3
    data['tp'] = 36e-3
    data['ap'] = 0.7
    data['tm'] = 36e-3 * data['ap']
    data['Brem'] = 1.23000000140598
    data['u0'] = 4e-7 * np.pi
    data['ur'] = 1.0997785406
    data['g'] = 1e-3
    data['rpm'] = data['Rm'] - data['Rr']
    data['rcoil'] = 15.9e-3
    data['tcoil'] = 7.8e-3
    data['Jrms'] = 4e6              
    data['b0'] = 2.5e-3
    data['ts'] = 12e-3
    data['p'] = 4
    data['SAP'] = data['rcoil'] * data['tcoil']
    data['Nsp'] = 100
    data['Pf'] = 1
    return data

data = Parameters()
z = torch.linspace(0, data['tp'] / 2 - 3e-3, 20)

In [ ]:
#PINN model

def Df2Tensor(df):
    array = np.array(df)
    tensor = torch.tensor(array, dtype=torch.float32)
    return tensor

def ToDataset(*args):
    return TensorDataset(*args)

def ToDataLoader(dataset, batchsize):
    return DataLoader(dataset, batchsize, shuffle=True)


class Net(nn.Module):
    def __init__(self, 
            input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4, 
            dropout1, dropout2, dropout3, dropout4):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim,hidden_layer1)
        self.layer2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.layer3 = nn.Linear(hidden_layer2,hidden_layer3)
        self.layer4 = nn.Linear(hidden_layer3,hidden_layer4)
        self.layer5 = nn.Linear(hidden_layer4,output_dim)

        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
        self.dropout3 = nn.Dropout(dropout3)
        self.dropout4 = nn.Dropout(dropout4)


    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = F.relu(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = F.relu(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        return x

class Metrics():
    def __init__(self, net, dataloader):
        dataset = dataloader.dataset
        self.features = dataset[:][0]
        self.labels = dataset[:][1]
        self.y_hat = torch.clamp(net(self.features), 1, float('inf'))
    def rmse(self):
        return torch.sqrt(F.mse_loss(self.y_hat, self.labels))
    def mae(self):
        return F.l1_loss(self.y_hat, self.labels)
    def smooth_mae(self):
        return F.smooth_l1_loss(self.y_hat, self.labels)
    def r2(self):
        # return r2_score(self.labels.detach().numpy(), self.y_hat.detach().numpy()) 
        SS_res = torch.sum(torch.square(self.labels-self.y_hat))
        SS_tot = torch.sum(torch.square(self.labels - torch.mean(self.labels)))
        r2 = 1 - SS_res / SS_tot
        return r2 

def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, std=0.01)


def DataConcat(Xtrain, Xtest, ytrain, ytest):
    train_df = [Xtrain, ytrain]
    test_df = [Xtest, ytest]
    train_data = pd.concat(train_df,axis=1)
    test_data = pd.concat(test_df,axis=1)
    return train_data, test_data


from torch.optim.lr_scheduler import StepLR

def train(net, dataloader, loss, num_epochs, lr, wd):
    net.train()

    optimizer = torch.optim.Adam(net.parameters(), lr = lr, weight_decay = wd)
    scheduler = StepLR(optimizer, step_size=num_epochs/3, gamma=0.3)

    for epoch in range(num_epochs):
        for X, y in dataloader:
            optimizer.zero_grad()
            
            y_pred = net(X)
            losses = []
            for i in range(X.shape[0]):
                b0 = X[i, 0]  
                ap = X[i, 1]  
                g = X[i, 2]  
                rcoil = X[i, 3]  
                rpm = X[i, 4]  
                
                y1_pred = y_pred[i, 0]  
                y2_pred = y_pred[i, 1]  
                y3_pred = y_pred[i, 2]  
                
            k1 = 2.2207169381003931e-10
            k2 = 1.1662311456953303e-08
            k3 = 7.414221187417784e-08
            k4 = 0.05587612460094443
            
            avgforce = (Avgforce(z, ap, rpm, g, rcoil, b0) - y1_pred)**2
            fluforce = (Fluforce(ap, rpm, g, rcoil, b0) - y2_pred)**2
            culoss = (CuLoss(g, rpm, rcoil) - y3_pred)**2
            
            PI_loss = k4*(k1 * avgforce + k2 * fluforce + k3 * culoss)

            losses.append(PI_loss)
            loss_tensor = torch.tensor(losses)
            
            l = loss(net(X), y) + loss_tensor
            l.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()



def NetEval(net, dataloader, num_epochs, loss, lr, wd):
    eval_list = []
    rmse, mae, r2 = [], [], []
    for epochs in range(num_epochs):
        net.train()
        train(net, dataloader, loss, num_epochs, lr, wd)

        net.eval()
        test_metrics = Metrics(net, dataloader)


        rmse.append(test_metrics.rmse().detach().item())
        mae.append(test_metrics.mae().detach().item())
        r2.append(test_metrics.r2().detach().item())
    return r2, mae, rmse

In [ ]:
input_dim, output_dim, hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4 = 5, 3, 120,60,30,15
num_epochs, lr, wd, batch_size = 1000, 0.01, 0, 54
dropout1, dropout2, dropout3, dropout4 = 0,0,0,0

loss = nn.MSELoss()

net = Net(input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4,
            dropout1, dropout2, dropout3, dropout4)
net.apply(init_weights)

Net(
  (layer1): Linear(in_features=5, out_features=120, bias=True)
  (layer2): Linear(in_features=120, out_features=60, bias=True)
  (layer3): Linear(in_features=60, out_features=30, bias=True)
  (layer4): Linear(in_features=30, out_features=15, bias=True)
  (layer5): Linear(in_features=15, out_features=3, bias=True)
  (dropout1): Dropout(p=0, inplace=False)
  (dropout2): Dropout(p=0, inplace=False)
  (dropout3): Dropout(p=0, inplace=False)
  (dropout4): Dropout(p=0, inplace=False)
)

In [ ]:
#PINN training
train(net, train_dataloader, loss, num_epochs, lr, wd)

In [ ]:
eval_epochs = 10
wd = 0
r2, mae, rmse = NetEval(net, test_dataloader, eval_epochs, loss, lr, wd)
PINN_r2_mean = np.mean(r2)
PINN_mae_mean = np.mean(mae)
PINN_rmse_mean = np.mean(rmse)
print('The average R² of PINN:{}, the average MAE:{}, the average RMSE:{}'.format(PINN_r2_mean, PINN_mae_mean, PINN_rmse_mean))

PINN r2平均值:0.9812212824821472, mae平均值:3.1910656929016112, rmse平均值:4.338607597351074


In [ ]:
#RF model
random_seed(0)
regressor = RandomForestRegressor(n_estimators = 100, max_depth=5)

regressor = MultiOutputRegressor(regressor)
regressor.fit(Xtrain, ytrain)

RF_scores = model_score(regressor, X, y, trainsize, testsize)

R2 = RF_scores['R2']
mae = RF_scores['mae']
rmse = RF_scores['rmse']
RF_R2_mean = np.mean(R2)
RF_mae_mean = np.mean(mae)
RF_rmse_mean = np.mean(rmse)
print('The average R² of RF:{}'.format(RF_R2_mean))
print('the average MAE:{}'.format(RF_mae_mean))
print('the average RMSE:{}'.format(RF_rmse_mean))

RF R2交叉验证平均值:0.8794056044943368
RF mae交叉验证平均值:3.68679542727138
RF rmse交叉验证平均值:5.895498904268028


In [ ]:
#SVM model
random_seed(0)
regressor = SVR(C=2)
regressor = MultiOutputRegressor(regressor)
regressor.fit(Xtrain, ytrain)

SVM_scores = model_score(regressor, X, y, trainsize, testsize)

R2 = SVM_scores['R2']
mae = SVM_scores['mae']
rmse = SVM_scores['rmse']
SVM_R2_mean = np.mean(R2)
SVM_mae_mean = np.mean(mae)
SVM_rmse_mean = np.mean(rmse)

print('The average R² of SVM:{}'.format(SVM_R2_mean))
print('the average MAE:{}'.format(SVM_mae_mean))
print('the average RMSE:{}'.format(SVM_rmse_mean))

SVM R2交叉验证平均值:0.7416618882275265
SVM mae交叉验证平均值:5.032454173805987
SVM rmse交叉验证平均值:8.587604946462989


In [ ]:
#AM data
Path = "AM_data.csv"

seed = 0
trainsize, testsize = 0.99,0.01

X, y = DataProcess(Path)
Xtrain, Xtest, ytrain, ytest = DataSplit(X,y, testsize, seed)

# type(Xtrain)
x1 = Xtrain.to_numpy()
y1 = ytrain.to_numpy()

train_dataset = ToDataset(Df2Tensor(Xtrain), Df2Tensor(ytrain))
test_dataset = ToDataset(Df2Tensor(Xtest), Df2Tensor(ytest))

batchsize = 54
train_dataloader1 = ToDataLoader(train_dataset, batchsize)
test_dataloader1 = ToDataLoader(test_dataset, batchsize)

In [ ]:
#Reset the network
class Net(nn.Module):
    def __init__(self, 
            input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4, 
            dropout1, dropout2, dropout3, dropout4):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim,hidden_layer1)
        self.layer2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.layer3 = nn.Linear(hidden_layer2,hidden_layer3)
        self.layer4 = nn.Linear(hidden_layer3,hidden_layer4)
        self.layer5 = nn.Linear(hidden_layer4,output_dim)

        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
        self.dropout3 = nn.Dropout(dropout3)
        self.dropout4 = nn.Dropout(dropout4)


    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = F.relu(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = F.relu(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        return x


class Metrics():
    def __init__(self, net, dataloader):
        dataset = dataloader.dataset
        self.features = dataset[:][0]
        self.labels = dataset[:][1]
        self.y_hat = torch.clamp(net(self.features), 1, float('inf'))
    def rmse(self):
        return torch.sqrt(F.mse_loss(self.y_hat, self.labels))
    def mae(self):
        return F.l1_loss(self.y_hat, self.labels)
    def smooth_mae(self):
        return F.smooth_l1_loss(self.y_hat, self.labels)
    def r2(self):
        SS_res = torch.sum(torch.square(self.labels-self.y_hat))
        SS_tot = torch.sum(torch.square(self.labels - torch.mean(self.labels)))
        r2 = 1 - SS_res / SS_tot
        return r2 

def train(net, dataloader, loss, num_epochs, lr, wd):
    net.train()
    

    optimizer = torch.optim.Adam(net.parameters(), lr = lr, weight_decay = wd)
    scheduler = StepLR(optimizer, step_size=num_epochs/3, gamma=0.3)

    for epoch in range(num_epochs):
        for X, y in dataloader:
            optimizer.zero_grad()
            
            l = loss(net(X), y) 
            l.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()


input_dim, output_dim, hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4 = 5, 3, 120,60,30,15
num_epochs, lr, wd, batch_size = 1000, 0.003, 0, 54
dropout1, dropout2, dropout3, dropout4 = 0,0.01,0.01,0.01

loss = nn.MSELoss()

net = Net(input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4,
            dropout1, dropout2, dropout3, dropout4)
net.apply(init_weights)

Net(
  (layer1): Linear(in_features=5, out_features=120, bias=True)
  (layer2): Linear(in_features=120, out_features=60, bias=True)
  (layer3): Linear(in_features=60, out_features=30, bias=True)
  (layer4): Linear(in_features=30, out_features=15, bias=True)
  (layer5): Linear(in_features=15, out_features=3, bias=True)
  (dropout1): Dropout(p=0, inplace=False)
  (dropout2): Dropout(p=0.01, inplace=False)
  (dropout3): Dropout(p=0.01, inplace=False)
  (dropout4): Dropout(p=0.01, inplace=False)
)

In [ ]:
#Pre-training and TL-DNN training
train(net, train_dataloader1, loss, num_epochs, lr, wd)
train(net, train_dataloader, loss, num_epochs, lr, wd)

In [ ]:
eval_epochs = 10
wd = 0
r2, mae, rmse = NetEval(net, test_dataloader, eval_epochs, loss, lr, wd)
DTNN_r2_mean = np.mean(r2)
DTNN_mae_mean = np.mean(mae)
DTNN_rmse_mean = np.mean(rmse)
print('The average R² of TL-DNN:{}, the average MAE:{}, the average RMSE:{}'.format(DTNN_r2_mean, DTNN_mae_mean, DTNN_rmse_mean))

DTNN r2平均值:0.9832298815250397, mae平均值:2.7834279775619506, rmse平均值:4.100418972969055


In [ ]:
#Reset the network and define TL-PINN
class Net(nn.Module):
    def __init__(self, 
            input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4, 
            dropout1, dropout2, dropout3, dropout4):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim,hidden_layer1)
        self.layer2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.layer3 = nn.Linear(hidden_layer2,hidden_layer3)
        self.layer4 = nn.Linear(hidden_layer3,hidden_layer4)
        self.layer5 = nn.Linear(hidden_layer4,output_dim)

        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
        self.dropout3 = nn.Dropout(dropout3)
        self.dropout4 = nn.Dropout(dropout4)


    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = F.relu(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = F.relu(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        return x


class Metrics():
    def __init__(self, net, dataloader):
        dataset = dataloader.dataset
        self.features = dataset[:][0]
        self.labels = dataset[:][1]
        self.y_hat = torch.clamp(net(self.features), 1, float('inf'))
    def rmse(self):
        return torch.sqrt(F.mse_loss(self.y_hat, self.labels))
    def mae(self):
        return F.l1_loss(self.y_hat, self.labels)
    def smooth_mae(self):
        return F.smooth_l1_loss(self.y_hat, self.labels)
    def r2(self):
        SS_res = torch.sum(torch.square(self.labels-self.y_hat))
        SS_tot = torch.sum(torch.square(self.labels - torch.mean(self.labels)))
        r2 = 1 - SS_res / SS_tot
        return r2 

def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, std=0.01)

def DataConcat(Xtrain, Xtest, ytrain, ytest):
    train_df = [Xtrain, ytrain]
    test_df = [Xtest, ytest]
    train_data = pd.concat(train_df,axis=1)
    test_data = pd.concat(test_df,axis=1)
    return train_data, test_data

from torch.optim.lr_scheduler import StepLR

def train(net, dataloader, loss, num_epochs, lr, wd):
    net.train()
    
    optimizer = torch.optim.Adam(net.parameters(), lr = lr, weight_decay = wd)
    scheduler = StepLR(optimizer, step_size=num_epochs/3, gamma=0.3)

    for epoch in range(num_epochs):
        for X, y in dataloader:
            optimizer.zero_grad()
            
            y_pred = net(X)
            losses = []
            for i in range(X.shape[0]):
                b0 = X[i, 0]  
                ap = X[i, 1]  
                g = X[i, 2]  
                rcoil = X[i, 3]  
                rpm = X[i, 4]  
                
                y1_pred = y_pred[i, 0]  
                y2_pred = y_pred[i, 1]  
                y3_pred = y_pred[i, 2]  
                
            k1 = 2.2207169381003931e-10
            k2 = 1.1662311456953303e-08
            k3 = 7.414221187417784e-08
            k4 = 0.05587612460094443
            
            avgforce = (Avgforce(z, ap, rpm, g, rcoil, b0) - y1_pred)**2
            fluforce = (Fluforce(ap, rpm, g, rcoil, b0) - y2_pred)**2
            culoss = (CuLoss(g, rpm, rcoil) - y3_pred)**2
            
            PI_loss = k4*(k1 * avgforce + k2 * fluforce + k3 * culoss)

            losses.append(PI_loss)
            loss_tensor = torch.tensor(losses)
            
            l = loss(net(X), y) + loss_tensor
            l.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()


In [ ]:
input_dim, output_dim, hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4 = 5, 3, 120,60,30,15
num_epochs, lr, wd, batch_size = 1000, 0.01, 0, 54
dropout1, dropout2, dropout3, dropout4 = 0,0,0,0

loss = nn.MSELoss()

net = Net(input_dim, output_dim, 
            hidden_layer1, hidden_layer2, hidden_layer3, hidden_layer4,
            dropout1, dropout2, dropout3, dropout4)
net.apply(init_weights)


Net(
  (layer1): Linear(in_features=5, out_features=120, bias=True)
  (layer2): Linear(in_features=120, out_features=60, bias=True)
  (layer3): Linear(in_features=60, out_features=30, bias=True)
  (layer4): Linear(in_features=30, out_features=15, bias=True)
  (layer5): Linear(in_features=15, out_features=3, bias=True)
  (dropout1): Dropout(p=0, inplace=False)
  (dropout2): Dropout(p=0, inplace=False)
  (dropout3): Dropout(p=0, inplace=False)
  (dropout4): Dropout(p=0, inplace=False)
)

In [ ]:
#Pre-training and TL-PINN training
train(net, train_dataloader1, loss, num_epochs, lr, wd)
train(net, train_dataloader, loss, num_epochs, lr, wd)

In [ ]:
eval_epochs = 10
wd = 0
r2, mae, rmse = NetEval(net, test_dataloader, eval_epochs, loss, lr, wd)
TPINN_r2_mean = np.mean(r2)
TPINN_mae_mean = np.mean(mae)
TPINN_rmse_mean = np.mean(rmse)
print('TL-PINN r2平均值:{}, mae平均值:{}, rmse平均值:{}'.format(TPINN_r2_mean, TPINN_mae_mean, TPINN_rmse_mean))

TL-PINN r2平均值:0.9860846757888794, mae平均值:2.5924966812133787, rmse平均值:3.7301127672195435


In [ ]:
print('The average R² of RF:{}, the average MAE:{}, the average RMSE:{}'.format(RF_R2_mean,RF_mae_mean,RF_rmse_mean))
print('The average R² of SVM:{}, the average MAE:{}, the average RMSE:{}'.format(SVM_R2_mean,SVM_mae_mean,SVM_rmse_mean))
print('The average R² of DNN:{}, the average MAE:{}, the average RMSE:{}'.format(DNN_r2_mean, DNN_mae_mean, DNN_rmse_mean))
print('The average R² of TL-DNN:{}, the average MAE:{}, the average RMSE:{}'.format(DTNN_r2_mean, DTNN_mae_mean, DTNN_rmse_mean))
print('The average R² of PINN:{}, the average MAE:{}, the average RMSE:{}'.format(PINN_r2_mean, PINN_mae_mean, PINN_rmse_mean))
print('The average R² of TL-PINN:{}, the average MAE:{}, the average RMSE:{}'.format(TPINN_r2_mean, TPINN_mae_mean, TPINN_rmse_mean))


all_evals = [[RF_R2_mean,RF_mae_mean,RF_rmse_mean],[SVM_R2_mean,SVM_mae_mean,SVM_rmse_mean],
             [DNN_r2_mean, DNN_mae_mean, DNN_rmse_mean],[DTNN_r2_mean, DTNN_mae_mean, DTNN_rmse_mean],
             [PINN_r2_mean, PINN_mae_mean, PINN_rmse_mean],[TPINN_r2_mean, TPINN_mae_mean, TPINN_rmse_mean]
             ]


RF r2平均值:0.8794056044943368, mae平均值:3.68679542727138, rmse平均值:5.895498904268028
SVM r2平均值:0.7416618882275265, mae平均值:5.032454173805987, rmse平均值:8.587604946462989
DNN r2平均值:0.9791010320186615, mae平均值:3.378544545173645, rmse平均值:4.57615213394165
DTNN r2平均值:0.9832298815250397, mae平均值:2.7834279775619506, rmse平均值:4.100418972969055
PINN r2平均值:0.9812212824821472, mae平均值:3.1910656929016112, rmse平均值:4.338607597351074
TL-PINN r2平均值:0.9860846757888794, mae平均值:2.5924966812133787, rmse平均值:3.7301127672195435
